# 思维树（ToT）示例

思维树（ToT）是一种链式结构，允许您使用思维树技术查询大型语言模型（LLM）。这是基于论文["Large Language Model Guided Tree-of-Thought"](https://arxiv.org/pdf/2305.08291.pdf)。

In [1]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=1, max_tokens=512, model="gpt-3.5-turbo-instruct")

/Users/harrisonchase/.pyenv/versions/3.9.1/envs/langchain/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
sudoku_puzzle = "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"  # 数独谜题
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"  # 数独解答
problem_description = f"""
{sudoku_puzzle}

- 这是一个4x4的数独谜题。
- *代表待填充的单元格。
- |字符用于分隔行。
- 每一步，用数字1-4替换一个或多个*。
- 任何行、列或2x2子网格中不能有重复的数字。
- 保留前一个有效思路中已知的数字。
- 每个思路可以是部分或最终解答。
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


## 基于规则的检查器

每个思考都会被思考检查器评估，并被赋予一个有效性类型：有效、无效或部分有效。一个简单的检查器可以基于规则。例如，在数独谜题的情况下，检查器可以检查谜题是有效的、无效的还是部分有效的。

在下面的代码中，我们实现了一个针对特定的4x4数独谜题的简单规则检查器。

In [3]:
import re
from typing import Tuple

from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity

# 创建一个名为MyChecker的类，继承自ToTChecker类
class MyChecker(ToTChecker):
    # 定义evaluate方法，接受problem_description和thoughts两个参数，并返回ThoughtValidity类型的值
    def evaluate(
        self, problem_description: str, thoughts: Tuple[str, ...] = ()
    ) -> ThoughtValidity:
        # 获取thoughts中的最后一个元素
        last_thought = thoughts[-1]
        # 去除空格和双引号
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        # 将"*"替换为"."，"|"替换为"\|"
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        # 如果数独解在clean_solution中，则返回VALID_FINAL
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        # 如果在clean_solution中找到regex_solution，则返回VALID_INTERMEDIATE
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        # 否则返回INVALID
        else:
            return ThoughtValidity.INVALID

只是对上面的MyChecker类进行测试：

In [4]:
# 代码注释

# 创建一个名为checker的MyChecker对象
checker = MyChecker()

# 使用assert语句来验证代码的正确性
# 调用checker对象的evaluate方法，并传入两个参数：空字符串和一个元组
# 元组中包含一个字符串，字符串的格式为"3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
# 调用evaluate方法的返回值与ThoughtValidity.VALID_INTERMEDIATE进行比较，如果相等则通过assert验证
assert (
    checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",))
    == ThoughtValidity.VALID_INTERMEDIATE
)

# 同上，验证第二个测试用例
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",))
    == ThoughtValidity.VALID_FINAL
)

# 同上，验证第三个测试用例
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",))
    == ThoughtValidity.VALID_INTERMEDIATE
)

# 同上，验证第四个测试用例
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",))
    == ThoughtValidity.INVALID
)

## 思维链树

初始化并运行思维链树，将最大交互次数 `k` 设置为 `30`，最大子思维数 `c` 设置为 `8`。

In [5]:
# 导入ToTChain类
from langchain_experimental.tot.base import ToTChain

# 创建ToTChain实例
tot_chain = ToTChain(
    llm=llm,  # 传入llm参数
    checker=MyChecker(),  # 传入MyChecker实例作为checker参数
    k=30,  # 设置k参数为30
    c=5,   # 设置c参数为5
    verbose=True,  # 设置verbose参数为True
    verbose_llm=False  # 设置verbose_llm参数为False
)

# 运行tot_chain的run方法，传入problem_description参数
tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.


/Users/harrisonchase/workplace/langchain/libs/langchain/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Thought: 3*,*,2|1*,3,*|*,1,*,3|4,*,*,1
Thought: 3*,1,2|1*,3,*|*,1,*,3|4,*,*,1
Thought: 3*,1,2|1*,3,4|*,1,*,3|4,*,*,1
Thought: 3*,1,2|1*,3,4|*,1,2,3|4,*,*,1
Thought: 3*,1,2|1*,3,4|2,1,*,3|4,*,*,1


Type <enum 'ThoughtValidity'> not serializable


Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|*,3,2,*|*,1,*,3|4,1,*,*
Thought: 3,2,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,2,*,2|1,*,3,*|1,1,*,3|4,1,*,*
Thought: 3,2,*,2|1,1,3,*|1,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,2,3,*|*,1,*,3|4,*,*,1
    Thought: 3,1,4,2|1,2,3,4|2,1,4,3|4,3,2,1
    Thought: 3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1

> Finished chain.


'3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'